In [186]:
# You need to create a prototype pricing model that can go through
# further validation and testing before being put into production.
# Eventually, this model may be the basis for fully automated quoting to clients,
# but for now, the desk will use it with manual oversight to explore options with the client.

# You should write a function that is able to use the data you created previously to price the contract.
# The client may want to choose multiple dates to inject and withdraw a set amount of gas,
# so your approach should generalize the explanation from before.
# Consider all the cash flows involved in the product.

# The input parameters that should be taken into account for pricing are:

# > Injection dates.
# > Withdrawal dates.
# > The prices at which the commodity can be purchased/sold on those dates.
# > The rate at which the gas can be injected/withdrawn.
# > The maximum volume that can be stored.
# > Storage costs.

# Write a function that takes these inputs and gives back the value of the contract.
# You can assume there is no transport delay and that interest rates are zero.
# Market holidays, weekends, and bank holidays need not be accounted for.
# Test your code by selecting a few sample inputs.

In [187]:
%reset -f
%load_ext autoreload
%autoreload 2
import os, sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from price_predict import load_data

ROOT="/content" # ROOT="/content/TaskOne"
data = os.path.join(ROOT, 'data/Nat_Gas.csv')
df = load_data(data)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/content/price_predict.py:35: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(fin, parse_dates=["Dates"],


In [188]:
def calc_storage_cost_and_drain(curr_vol: int, drain_time: int, gas_rate_tput: int, storage_cost: float) -> tuple:
    """
    as long as tank is not empty, add a day's cost, reduce tank volume
    return tuple of (vol_sold, curr_vol, total_cost)
    """
    total_cost = 0.0 # amount that was paid in storage costs
    # calculate P/L selling the contract outside this function
    # with vol_sold * sale_price
    vol_sold = curr_vol
    # drain the tank
    for i in range(drain_time):
        total_cost = total_cost + storage_cost
        if curr_vol >= gas_rate_tput:
            curr_vol -= gas_rate_tput
        else:
            curr_vol = 0

    vol_sold -= curr_vol
    return vol_sold, curr_vol, total_cost

In [189]:
def calc_volume_and_buy(curr_vol: int, fill_time: int, gas_rate_tput: int, max_vol: int) -> tuple:
    """
    fill the tank as long as it is not full
    return tuple of (vol_bought, curr_vol, total_cost)
    """
    total_cost = 0 # amount that could be bought * buy_price
    vol_bought = 0
    # fill the tank
    for i in range(fill_time):
        if curr_vol + gas_rate_tput <= max_vol:
            vol_bought += gas_rate_tput
            curr_vol += gas_rate_tput
        else:
            vol_bought += max_vol - curr_vol
            curr_vol = max_vol

    return vol_bought, curr_vol, total_cost

In [190]:
from price_predict import run_holt_winters

def price_contract(inj_dates: list, wth_dates: list, gas_rate_tput: int, max_vol: int, storage_cost: float) -> float:
    """
    Inputs:
    inj_dates:     list of injection dates  (sort earliest to latest)
    wth_dates:     list of withdrawal dates (sort earliest to latest)
    gas_rate_tput: rate at which the gas can be injected/withdrawn
                        defining this as daily rate
    max_vol:       maximum volume that can be stored
    storage_cost:  cost of storage
                        going to define as cost per day to use tank of capacity max_vol

    Calculate:
    # The prices at which the commodity can be purchased/sold on those dates.
    buy_price_i:  price at which the commodity can be purchased on inj_dates[i]
    sell_price_j: price at which the commodity can be sold on      wth_dates[j]
    # calculate via TaskOne : Holt-Winters price prediction model

    Outputs:
    contract_value: value of the contract
    """
    # exit if Injection dates are empty
    if not inj_dates:
        print("No commodity buying occured. Exiting")
        return 0

    # convert Injection dates, Withdrawal dates to pd.to_datetime()
    inj_dates = np.array([pd.to_datetime(x, format='%Y-%m-%d') for x in inj_dates])
    wth_dates = np.array([pd.to_datetime(x, format='%Y-%m-%d') for x in wth_dates])

    # sort Injection dates, Withdrawal dates earliest to latest
    inj_dates = np.sort(inj_dates)
    wth_dates = np.sort(wth_dates)

    # validate if dates are in bounds
    # we don't need a buy before every sell
    # we do need gas in the tank if we are to sell though or else skip sell date?

    # discard any wth_date_j that occurs before first inj_date_i
    first_buy = inj_dates[0]
    trim_idx = np.searchsorted(wth_dates, first_buy)
    wth_dates = wth_dates[trim_idx:]

    # no issue if no selling occurs, will just store a lot and cost interest
    # make sure not to overflow the tank
    contract_value = 0.0
    curr_tank_vol = 0
    storage_cost_total = 0.0

    # assume gas price locked in on buy day even if it takes longer to fill / withdraw

    # naive way first - buy, sell, buy, sell, buy, sell etc
    count = 0
    prev_sell_date_j = None
    sell_price_j = 0.0
    for inj_date_i, wth_date_j in zip(inj_dates, wth_dates):

        # calc draining oil from previous sell date
        # we need to see how much we can drain before purchase occurs
        # FIXME: handle overlap in future
        if count > 0:
            drain_time = (inj_date_i - prev_sell_date_j).days
            # calculate the storage cost of the draining supply
            vol_sold, curr_tank_vol, store_cost = calc_storage_cost_and_drain(
                curr_tank_vol, drain_time, gas_rate_tput, storage_cost
            )
            storage_cost_total += store_cost
            # P/L from the amount we could sell
            contract_value += sell_price_j * vol_sold
            print(f"The contract is worth ${contract_value:.2f}\n")

        time_duration = (wth_date_j - inj_date_i).days
        print(f'inj_date_i = {inj_date_i}')
        print(f'wth_date_j = {wth_date_j}')
        print(f'time_duration = {time_duration}')

        # fill up the tank
        # 0 <= {vol_actual, curr_tank_vol} <= max_vol
        vol_bought, curr_tank_vol, buy_cost = calc_volume_and_buy(
            curr_tank_vol, time_duration, gas_rate_tput, max_vol
        )
        print(f'vol_bought = {vol_bought}')
        print(f'curr_tank_vol = {curr_tank_vol}')

        # retrieve buy_price_i and sell_price_i from Holt-Winters price pred. model
        buy_price_i = run_holt_winters(inj_date_i, df)
        sell_price_j = run_holt_winters(wth_date_j, df)
        print(f'buy_price_i = {buy_price_i}')
        print(f'sell_price_j = {sell_price_j}')

        # can calc cost of buying with vol bought now
        # for selling, do at beginning of next loop or after looping
        # to see draining time and costs
        contract_value -= buy_price_i * vol_bought
        # print(f'contract_value = {contract_value}')
        print(f'storage_cost_total = ${storage_cost_total:.2f}\n')

        count += 1
        prev_sell_date_j = wth_date_j

    # drain entire tank
    if prev_sell_date_j:
        sell_price = run_holt_winters(prev_sell_date_j, df)
        # calc storage cost during final drainage
        drain_time = int(np.ceil(curr_tank_vol / gas_rate_tput))

        # calculate the storage cost of the draining supply
        vol_sold, curr_tank_vol, store_cost = calc_storage_cost_and_drain(
            curr_tank_vol, drain_time, gas_rate_tput, storage_cost
        )
        contract_value += sell_price * vol_sold
        storage_cost_total += store_cost

    # subtract storage costs from the contract at the end
    print(f'Total storage costs: ${storage_cost_total:.2f}')
    contract_value -= storage_cost_total
    print(f"The contract is worth ${contract_value:.2f}\n")
    return contract_value


In [192]:
# Sample Inputs 0
# testing all in bounds, buy then sell without overlap
inj_dates = ["2020-10-11", "2021-04-05", "2022-08-09", "2023-10-01", "2024-09-09"]
wth_dates = ["2021-02-02", "2022-06-27", "2023-06-06", "2024-01-24", "2024-10-10"]
gas_rate_tput = 1000 # per day
max_vol = 100000     # tank total
storage_cost = 100   # per day
contract_value = price_contract(inj_dates, wth_dates, gas_rate_tput, max_vol, storage_cost)
# print(f"The contract is worth ${contract_value:.2f}\n")

inj_date_i = 2020-10-11 00:00:00
wth_date_j = 2021-02-02 00:00:00
time_duration = 114
vol_bought = 100000
curr_tank_vol = 100000
On 2020-10-11 00:00:00, the expected gas price is $10.10
On 2021-02-02 00:00:00, the expected gas price is $10.90
buy_price_i = 10.1
sell_price_j = 10.9
storage_cost_total = $0.00

The contract is worth $-334200.00

inj_date_i = 2021-04-05 00:00:00
wth_date_j = 2022-06-27 00:00:00
time_duration = 448
vol_bought = 62000
curr_tank_vol = 100000
On 2021-04-05 00:00:00, the expected gas price is $10.40
On 2022-06-27 00:00:00, the expected gas price is $10.40
buy_price_i = 10.4
sell_price_j = 10.4
storage_cost_total = $6200.00

The contract is worth $-531800.00

inj_date_i = 2022-08-09 00:00:00
wth_date_j = 2023-06-06 00:00:00
time_duration = 301
vol_bought = 43000
curr_tank_vol = 100000
On 2022-08-09 00:00:00, the expected gas price is $10.40
On 2023-06-06 00:00:00, the expected gas price is $10.90
buy_price_i = 10.4
sell_price_j = 10.9
storage_cost_total = $10500